In [1]:
clean_up=True # removes gams-related files in work-folder if true
%run StdPackages.ipynb
os.chdir(directory['py'])
import techdata_to_tree, sys, ShockFunction
import abatement_EOP_new as abatement
os.chdir(directory['curr'])

*Specify settings:*

In [2]:
name = 'A4'
# catalogue = 'dors_2'
catalogue = 'C2'
f = {'tech': directory['data']+f"/techdata_{catalogue}.xlsx",
     'other': directory['data']+f"/Othdata_{catalogue}.xlsx",
     'IOdata': directory['data']+f"/IOdata_{catalogue}.xlsx"}
s = ['inputdisp','endofpipe','inputprices']
m = ['ID','EOP'] 
e = {'M': 'vars','M_sets': 'sets'} # dictionary used to load emissions data
other_trees = {'ID_Y': {'s': 'Y', 'read_data': {'Y': 'vars'}, 'Q2P': 'Q2P'}} # the key = tree name, s = sheet that specifies tree structure, read_data (if not None) specifies how to load data.

In [3]:
type_f = {'ID_EC': 'CES_norm',
          'ID_CU': 'MNL',
          'ID_TU': 'CET_norm',
          'ID_TX': 'CES',
          'ID_BU': 'linear_out',
          'ID_BX': 'CES',
          'EOP_CU': 'MNL',
          'EOP_TU': 'CET_norm',
          'EOP_TX': 'CES'}

*Set up nesting tree:*

In [4]:
%run ATree.ipynb

In [5]:
dbs = {}

## 1: Model without EOP

*Initialize without EOP:*

In [6]:
gm = abatement.abate(nts=nts,tech=tech,work_folder=directory['work'],use_EOP=False,**{'data_folder':directory['gams'],'name': name})
gm.add_aliases([('n','nnnn'),('n','nnnnn')])
gm.model.database.update_all_sets(clean_up=False) 

*Provide initial values:*

In [7]:
gm.initialize_variables_leontief()
gm.initialize_variables(**{'check_variables': True})

*Add some extra options:*

In [8]:
with open(os.getcwd()+'\\conopt_ExtraPivot.opt',"r") as file:
    kwargs_mi = {'opt_file': file.read()}

### 1.1: Solve with leontief values:

In [9]:
gm.write_and_run(name='ID',kwargs_mi=kwargs_mi)
dbs['ID'] = gm.model_instances['ID'].out_db

*Merge into starting values:*

In [10]:
gm.model.database.merge_dbs(gm.model.database,dbs['ID'],'second')

*Add new target smoothing values:*

In [11]:
db_star = DataBase.GPM_database()
db_star['eta'] = pd.Series(-5, index = gm.get('eta').index, name = 'eta')
db_star['sigma'] = pd.Series(10, index = gm.get('kno_ID_CU'),name='sigma').combine_first(
                   pd.Series(0.5, index = gm.get('kno_ID_BX'),name='sigma')).combine_first(
                   pd.Series(0.5, index = gm.get('kno_ID_Y'), name = 'sigma'))
db_star['mu'] = pd.Series(1, index = gm.get('kno_ID_CU'),name='mu')

*Solve smoothly:*

In [12]:
gm.model_instances['ID'].solve_sneakily(db_star=db_star,kwargs_shock={'n_steps':100})

{'Modelstat': 16.0, 'Solvestat': 1.0}

*Write back data:*

In [13]:
gm.model.database.merge_dbs(gm.model.database,dbs['ID'],'second')

### 1.2: Calibrate

In [14]:
gm.reset_settings()
gm.setstate('ID_calibrate',init=False)

*Load IO data:*

In [28]:
gm.model.database['mu'].rctree_pd(gm.get('ID_mu_endoincalib'))

n                        nn         
C0_EH                    EH             0.580000
C_EH_1                   EH             0.120000
C_EH_2                   EH             0.180000
C_EH_3                   EH             0.120000
EH                       out1           0.500000
ID_t1_K                  ID_t1          1.600000
ID_t1_electricity        ID_t1          1.000000
ID_t1_oil                ID_t1          0.400000
ID_t2_K                  ID_t2          5.000000
ID_t2_electricity        ID_t2          1.000000
U0_ID_C0_EH              C0_EH          1.000000
                         basetech_EH    1.000000
U0_ID_C_EH_1             C_EH_1         0.900000
                         basetech_EH    1.000000
U0_ID_C_EH_2             C_EH_2         0.866667
                         basetech_EH    1.000000
U0_ID_C_EH_3             C_EH_3         0.966667
                         basetech_EH    1.000000
U_ID_t1_1                C_EH_1         0.100000
                         ID_t1  

In [15]:
db = gm.add_calib_data(f["IOdata"], currapp=False)